In [1]:
from __future__ import print_function
import sys, json, os
import subprocess
import multiprocessing as mp
import numpy as np
from array import array

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import ROOT
from ROOT import gROOT, gStyle, gDirectory, gPad

import constants
import crv_event
import crv_spill
import utils
import root_utils
import geometry
import geometry_constants
import filepath

topdir = os.path.dirname(os.path.abspath("__file__"))
analysis_dir = os.path.join(topdir,"analysis_root_files")

# fileListM127 = filepath.getfilelist(["crvaging001", "crvaging007"],"recoROOT",3)[0]
# fileListM127 = filepath.getfilelist(["crvaging001", "crvaging007"],"recoROOT",4)[0]
fileListM127 = filepath.getfilelist(["crvaging001", "crvaging007"],"recoROOT",5)[0]
fileListM127.sort(key=lambda x:(filepath.filenameparser(x,'run'), filepath.filenameparser(x,'subrun')))
# rootNameM127 = os.path.join(analysis_dir,"crvaging_module127_v003.root")
# rootNameM127 = os.path.join(analysis_dir,"crvaging_module127_v004.root")
rootNameM127 = os.path.join(analysis_dir,"crvaging_module127_v005.root")
# fileListM168 = filepath.getfilelist(["crvaging003", "crvaging004", "crvaging005", "crvaging006"],"recoROOT",3)[0]
# fileListM168 = filepath.getfilelist(["crvaging003", "crvaging004", "crvaging005", "crvaging006"],"recoROOT",4)[0]
fileListM168 = filepath.getfilelist(["crvaging003", "crvaging004", "crvaging005", "crvaging006"],"recoROOT",5)[0]
fileListM168.sort(key=lambda x:(filepath.filenameparser(x,'run'), filepath.filenameparser(x,'subrun')))
# rootNameM168 = os.path.join(analysis_dir,"crvaging_module168_v003.root")
# rootNameM168 = os.path.join(analysis_dir,"crvaging_module168_v004.root")
rootNameM168 = os.path.join(analysis_dir,"crvaging_module168_v005.root")
taskList = [(fileListM127, rootNameM127), (fileListM168, rootNameM168)]

def processFiles(fileList,rootName):
    analysisFile = ROOT.TFile(rootName, "RECREATE")
    analysisFile.mkdir("spectrum")
    analysisFile.cd()
    spill_summary_tree = ROOT.TTree('spill_summary','spill_summary')
    subrun_summary_tree = ROOT.TTree('subrun_summary','subrun_summary')
    spill_summary_entry_dict = root_utils.treeInitialization(spill_summary_tree, utils.spill_summary_branch_list)
    subrun_summary_entry_dict = root_utils.treeInitialization(subrun_summary_tree, utils.subrun_summary_branch_list)

    for filename in fileList:
        utils.processSubRun(filename, analysisFile, spill_summary_tree, subrun_summary_tree, spill_summary_entry_dict, subrun_summary_entry_dict, 
                            True, '== 0x0', False, "110", False) 
                            # applyManualDQC = True, dqmFilter = '== 0x0', dqmVerbose = False, spectrumTempCorrection = "110", weightedAvg = False
        
    analysisFile.cd()
    spill_summary_tree.Write()
    subrun_summary_tree.Write()
    analysisFile.Close()
    return

nWorker = 2
p = mp.Pool(processes=nWorker)
for taskspec in taskList:
    p.apply_async(processFiles, (taskspec[0],taskspec[1]))
p.close()
p.join()

Welcome to JupyROOT 6.28/04
Reading file:Reading file:  rec.mu2e.CRV_wideband_cosmics.CRVWB-000-005-000.000066_000.rootrec.mu2e.CRV_wideband_cosmics.CRVWB-000-005-000.001059_000.root

Run1059_000 processed 200/693 spills...
Run0066_000 processed 200/1000 spills...
Run1059_000 processed 400/693 spills...
Run0066_000 processed 400/1000 spills...
Run1059_000 processed 600/693 spills...
Loading calibration information from rec.mu2e.CRV_wideband_cosmics.CRVWB-000-005-000.001059_000.root
Reading file: rec.mu2e.CRV_wideband_cosmics.CRVWB-000-005-000.001080_000.root
Run0066_000 processed 600/1000 spills...
Run1080_000 processed 200/499 spills...
Run0066_000 processed 800/1000 spills...
Run1080_000 processed 400/499 spills...
Loading calibration information from rec.mu2e.CRV_wideband_cosmics.CRVWB-000-005-000.000066_000.root
Reading file: rec.mu2e.CRV_wideband_cosmics.CRVWB-000-005-000.000066_001.root
Loading calibration information from rec.mu2e.CRV_wideband_cosmics.CRVWB-000-005-000.001080_00

In [2]:
from __future__ import print_function
import sys, json, os
import subprocess
import multiprocessing as mp
import numpy as np
from array import array

lock = mp.Manager().Lock()

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import ROOT
from ROOT import gROOT, gStyle, gDirectory, gPad

import constants
import crv_event
import crv_spill
import utils
import root_utils
import geometry
import geometry_constants
import filepath
import manualDQC

gROOT.Reset()
gROOT.SetBatch(1)
gROOT.ProcessLine( "gErrorIgnoreLevel = 1001;")
gStyle.SetOptStat(111110)
gStyle.SetOptFit(0)
gStyle.SetLineScalePS(0.3)

topdir = os.path.dirname(os.path.abspath("__file__"))
analysis_dir = os.path.join(topdir,"analysis_root_files")

#dataver = "v003"
#dataver = "v004"
dataver = "v005"

firstPass = False

rootNameM127 = os.path.join(analysis_dir,"crvaging_module127_"+dataver+".root")
pdfNameM127 = os.path.join(analysis_dir,"crvaging_module127_"+dataver+".pdf")
FEBList127 = [0, 1]
rootNameM168 = os.path.join(analysis_dir,"crvaging_module168_"+dataver+".root")
pdfNameM168 = os.path.join(analysis_dir,"crvaging_module168_"+dataver+".pdf")
FEBList168 = [1, 2]
taskList = [(rootNameM127, pdfNameM127, FEBList127), (rootNameM168, pdfNameM168, FEBList168)]

refitList = []
if dataver == "v004":
    refitList = [(168,1146,0,2,13)] # 004

fitThres = 2400
# fitmin = [20., 40]
# fitmax = [100., 200]

def LandauGaussFunction(x, par):
    # From recoCrv::double LandauGaussFunction(double *x, double *par) 
    # From $ROOTSYS/tutorials/fit/langaus.C
    # Fit parameters:
    # par[0]=Width (scale) parameter of Landau density
    # par[1]=Most Probable (MP, location) parameter of Landau density
    # par[2]=Total area (integral -inf to inf, normalization constant)
    # par[3]=Width (sigma) of convoluted Gaussian function
    
    # In the Landau distribution (represented by the CERNLIB approximation),
    # the maximum is located at x=-0.22278298 with the location parameter=0.
    # This shift is corrected within this function, so that the actual
    # maximum is identical to the MP parameter.

    #  Numeric constants
    invsq2pi = 0.3989422804014   # (2 pi)^(-1/2)
    mpshift  = -0.22278298       # Landau maximum location

    # Control constants
    np = 100.0      # number of convolution steps
    sc =   5.0      # convolution extends to +-sc Gaussian sigmas

    # Variables
    xx = 0.0
    mpc = 0.0
    fland = 0.0
    sum_val = 0.0
    xlow = 0.0
    xupp = 0.0
    step = 0.0
    
    # MP shift correction
    mpc = par[1] - mpshift * par[0]

    # Range of convolution integral
    xlow = x[0] - sc * par[3]
    xupp = x[0] + sc * par[3]
    step = (xupp - xlow) / np

    # Convolution integral of Landau and Gaussian by sum
    for i in range(1, int(np/2) + 1):
        xx = xlow + (i - 0.5) * step
        fland = ROOT.TMath.Landau(xx, mpc, par[0]) / par[0]
        sum_val += fland * ROOT.TMath.Gaus(x[0], xx, par[3])

        xx = xupp - (i - 0.5) * step
        fland = ROOT.TMath.Landau(xx, mpc, par[0]) / par[0]
        sum_val += fland * ROOT.TMath.Gaus(x[0], xx, par[3])

    return par[2] * step * sum_val * invsq2pi / par[3]

def evaluateModule(rootname, pdfname, FEBList):
    
    moduleNum = int(pdfname[-12:-9])
    
    fC = ROOT.TCanvas("c%i"%moduleNum, "c%i"%moduleNum, 1600, 900)
    fC.Print(pdfname+"[", "pdf")
           
    fFile = ROOT.TFile(rootname, "READ")
    
    fROOTtempStorage = None
    if firstPass:
        fROOTtempStorage = ROOT.TFile(rootname[:-5]+"_tempStorage.root", "RECREATE")
    else:
        fROOTtempStorage = ROOT.TFile(rootname[:-5]+"_tempStorage.root", "READ")
    subrun_summary_tree = fFile.Get('subrun_summary')
    nSubruns = subrun_summary_tree.GetEntries()
    fFile.cd()
    
    t0 = None
    
    gStyle.SetOptStat(111110)
    fGraphAging = [[None]*geometry_constants.nChannelPerFEB for i in range(len(FEBList))]
    fGraphCMBTemp = [[None]*geometry_constants.nChannelPerFEB for i in range(len(FEBList))]
    fGrpahSPECorrected = [[None]*geometry_constants.nChannelPerFEB for i in range(len(FEBList))]
    SPECorrected_FEBVertical = [[[] for j in range(geometry_constants.nChannelPerFEB)] for i in range(len(FEBList))]
    fHistAging = [None]*len(FEBList)
    
    for ii in range(len(FEBList)):
        iFEB = FEBList[ii]
        fHistAging[ii] = ROOT.TH1F("hist_aging_rate_%i_%i"%(moduleNum, iFEB),
                                   "Module %i aging rate; aging rate [%%]; count"%(moduleNum),
                                   50, 0., 10.)
        for iCh in range(geometry_constants.nChannelPerFEB):
            fGraphAging[ii][iCh] = ROOT.TGraphErrors()
            fGraphAging[ii][iCh].SetName("fGraphAging_Module%i_FEB%i_Ch%i"%(moduleNum,iFEB,iCh))
            fGraphAging[ii][iCh].SetTitle("Module%i FEB%i Ch%i corrected light yield; Time [year]; Light yield [PE]"%(moduleNum,iFEB,iCh))
            fGraphAging[ii][iCh].SetMarkerStyle(1)
            fGraphAging[ii][iCh].SetMarkerColor(4)
            fGraphAging[ii][iCh].SetLineColor(4)
            fGraphAging[ii][iCh].SetLineWidth(3)
            fGraphAging[ii][iCh].SetMarkerSize(1)

            fGraphCMBTemp[ii][iCh] = ROOT.TGraph()
            fGraphCMBTemp[ii][iCh].SetName("fGraphCMBTemp_Module%i_FEB%i_Ch%i"%(moduleNum,iFEB,iCh))
            fGraphCMBTemp[ii][iCh].SetTitle("Module%i FEB%i Ch%i CMB temperature; Time [year]; Temperature [degC]"%(moduleNum,iFEB,iCh))
            fGraphCMBTemp[ii][iCh].SetMarkerStyle(21)
            fGraphCMBTemp[ii][iCh].SetMarkerColor(4)
            fGraphCMBTemp[ii][iCh].SetLineColor(4)
            fGraphCMBTemp[ii][iCh].SetLineWidth(3)
            fGraphCMBTemp[ii][iCh].SetMarkerSize(1)

            fGrpahSPECorrected[ii][iCh] = ROOT.TGraph()
            fGrpahSPECorrected[ii][iCh].SetName("fGrpahSPECorrected_Module%i_FEB%i_Ch%i"%(moduleNum,iFEB,iCh))
            fGrpahSPECorrected[ii][iCh].SetTitle("Module%i FEB%i Ch%i corrected SPE; Time [year]; Corrected SPE [ADC*ns/PE]"%(moduleNum,iFEB,iCh))
            fGrpahSPECorrected[ii][iCh].SetMarkerStyle(21)
            fGrpahSPECorrected[ii][iCh].SetMarkerColor(4)
            fGrpahSPECorrected[ii][iCh].SetLineColor(4)
            fGrpahSPECorrected[ii][iCh].SetLineWidth(3)
            fGrpahSPECorrected[ii][iCh].SetMarkerSize(1)
    
    hist_calibAdj_sigma = ROOT.TH1F("hist_calibAdj_sigma", "#sigma of Adjusted SPE (Vertical FEB Orientation); #sigma_{SPE} [ADC*ns/PE]; counts", 40, 0., 8.)
    hist_calibAdj_sigma_mu = ROOT.TH1F("hist_calibAdj_sigma_mu", "#sigma/#mu of Adjusted SPE (Vertical FEB Orientation); #sigma_{SPE}/#mu_{SPE}; counts", 40, 0., 0.02)
    tsEpochVeticalFEB = 1664928000 # 2022-10-5, when rack was installed
    
    for iSubrun in range(nSubruns):
        subrun_summary_tree.GetEntry(iSubrun)
        runNum = subrun_summary_tree.runNum
        subrunNum = subrun_summary_tree.subrunNum
    
        tsEpochStart = subrun_summary_tree.tsEpochStart
        tsEpochEnd = subrun_summary_tree.tsEpochEnd
        tYearRaw = (tsEpochEnd+tsEpochStart)/2./60./60./24./365.
        tYear_err = (tsEpochEnd-tsEpochStart)/2./60./60./24./365.
        
        if t0 is None:
            t0 = tYearRaw
            tYear = 0.
        else:
            tYear = tYearRaw-t0
            
        isVertical = True if tsEpochStart > tsEpochVeticalFEB else False
        
        lock.acquire()
        print("Processing Run%04i_%03i"%(runNum, subrunNum))
        if tYear_err>0.1:
            print(tsEpochStart/2./60./60./24./365., tsEpochEnd/2./60./60./24./365., tYear, tYear_err)
        sys.stdout.flush()
        lock.release()
        
        myfunc = [[None]*geometry_constants.nChannelPerFEB for i in range(len(FEBList))]
        fitfunc = [[None]*geometry_constants.nChannelPerFEB for i in range(len(FEBList))]
                
        for ii in range(len(FEBList)):
            iFEB = FEBList[ii]
            if (runNum, subrunNum) in manualDQC.manual_CMB_temp_exclusion_dict:
                if iFEB in manualDQC.manual_CMB_temp_exclusion_dict[(runNum, subrunNum)]:
                    continue
            
            for iCh in range(geometry_constants.nChannelPerFEB):
                fC.Clear()                
                index = iFEB * geometry_constants.nChannelPerFEB + iCh
                                  
                tSpectrumName = subrun_summary_tree.spectrumCorrected[index]
                
                tSpectrum = None
                if firstPass:
                    tSpectrum = fFile.Get(tSpectrumName).Clone()
                else:
                    tSpectrum = fROOTtempStorage.Get(tSpectrumName.split('/')[1]).Clone()
                
                maxbin = 0
                secondmaxbin = 0
                thirdmaxbin = 0
                maxbinContent = 0
                nHits = 0
                for ibin in range(6, tSpectrum.GetNbinsX()): # ignore first few bins that are noise peaks
                    binContent = tSpectrum.GetBinContent(ibin)
                    nHits += binContent
                    if binContent > maxbinContent:
                        thirdmaxbin=secondmaxbin
                        secondmaxbin=maxbin
                        maxbin=ibin
                        maxbinContent=binContent
                
                tSpectrum.SetMaximum(maxbinContent*1.2)
                tSpectrum.Draw()
                tPT = ROOT.TPaveText(.7, .4, .9, .72, "NDC")
                tPT.SetFillColor(0)
                tPT.AddText("%i hits #geq 5 PE"%nHits)
                
                peakrough = (maxbin+secondmaxbin+thirdmaxbin)/3.
                fitmin = peakrough/1.5
                fitmax = peakrough*2.4
                           
                # fit landgaus
                if nHits > fitThres:
                    
                    funcName = None                    
                    if firstPass or (moduleNum,runNum,subrunNum,iFEB,iCh) in refitList:
                        
                        myfunc[ii][iCh] = LandauGaussFunction
                        funcName = "func_Run%06i_%03i_FEB%02i_Ch%02i"%(runNum,subrunNum,iFEB,iCh)
                        
                        if (not firstPass) and ((moduleNum,runNum,subrunNum,iFEB,iCh) in refitList):
                            print("Refitting Run%06i_%03i_FEB%02i_Ch%02i"%(runNum,subrunNum,iFEB,iCh))
                            tempfunc = fROOTtempStorage.Get(funcName).Clone()
                            fitmin = tempfunc.GetParameter(1)/1.5
                            fitmax = tempfunc.GetParameter(1)*2.4
                            maxbin = tempfunc.GetParameter(1)
                        
                        fitfunc[ii][iCh] = ROOT.TF1(funcName, myfunc[ii][iCh], fitmin, fitmax, 4)
                        fitfunc[ii][iCh].SetParameter(0, 3.5 if moduleNum == 127 else 4.5)
                        fitfunc[ii][iCh].SetParameter(1, float(maxbin))
                        fitfunc[ii][iCh].SetParameter(2, float(nHits))
                        fitfunc[ii][iCh].SetParameter(3, 8. if moduleNum == 127 else 10.)
                        fitfunc[ii][iCh].SetParName(0, "Width")
                        fitfunc[ii][iCh].SetParName(1, "MP")
                        fitfunc[ii][iCh].SetParName(2, "Area")
                        fitfunc[ii][iCh].SetParName(3, "#sigma_{gaus}")
                        fitfunc[ii][iCh].SetParLimits(0, 1., 30.)
                        fitfunc[ii][iCh].SetParLimits(1, maxbin-30., maxbin+30.)
                        fitfunc[ii][iCh].SetParLimits(2, 0.01*nHits, 100.*nHits)
                        fitfunc[ii][iCh].SetParLimits(3, 0.5, 30.)
                        # print("Fitting FEB%i Ch%i"%(iFEB, iCh))
                        frp = tSpectrum.Fit(fitfunc[ii][iCh], "QRSL0")
                    else:
                        funcName = "func_Run%06i_%03i_FEB%02i_Ch%02i"%(runNum,subrunNum,iFEB,iCh)
                        fitfunc[ii][iCh] = fROOTtempStorage.Get(funcName).Clone()
                    
                    fitfunc[ii][iCh].SetLineColor(2)
                    fitfunc[ii][iCh].SetLineWidth(1)
                    fitfunc[ii][iCh].DrawClone("SAME")
                    
                    tPT.AddText("time in year %.3f"%(tYear))
                    tPT.AddText("#chi^{2}/NDF = %.3f/%i"%(fitfunc[ii][iCh].GetChisquare(), fitfunc[ii][iCh].GetNDF()))
                    tPT.AddText("Width = %.3f #pm %.3f"%(fitfunc[ii][iCh].GetParameter(0),fitfunc[ii][iCh].GetParError(0)))
                    tPT.AddText("MP = %.3f #pm %.3f"%(fitfunc[ii][iCh].GetParameter(1),fitfunc[ii][iCh].GetParError(1)))
                    tPT.AddText("Area = %.3f #pm %.3f"%(fitfunc[ii][iCh].GetParameter(2),fitfunc[ii][iCh].GetParError(2)))
                    tPT.AddText("#sigma_{gaus} = %.3f #pm %.3f"%(fitfunc[ii][iCh].GetParameter(3),fitfunc[ii][iCh].GetParError(3)))
                    
                    if fitfunc[ii][iCh].GetNDF()>0.: # ignore failed fittings
                        iPoint = fGraphAging[ii][iCh].GetN()
                        fGraphAging[ii][iCh].SetPoint(iPoint,tYear,fitfunc[ii][iCh].GetParameter(1))
                        fGraphAging[ii][iCh].SetPointError(iPoint,tYear_err,fitfunc[ii][iCh].GetParError(1))
                    else: 
                        print("Bad fit %i Run%04i_%03i FEB%i Ch%i"%(moduleNum, runNum, subrunNum, iFEB, iCh))
                
                tPT.DrawClone("SAME")
                fC.Print(pdfname, "Title: Run%04i_%03i FEB%i Ch%i"%(runNum, subrunNum, iFEB, iCh))
                
                fGraphCMBTemp[ii][iCh].SetPoint(fGraphCMBTemp[ii][iCh].GetN(),tYear,subrun_summary_tree.temperatureCMB[index])
                fGrpahSPECorrected[ii][iCh].SetPoint(fGrpahSPECorrected[ii][iCh].GetN(),tYear,subrun_summary_tree.calibAdj[index])
                
                if isVertical:
                    if subrun_summary_tree.calibAdj[index] > 0:
                        SPECorrected_FEBVertical[ii][iCh].append(subrun_summary_tree.calibAdj[index])
                    
                if firstPass:
                    fROOTtempStorage.cd()
                    if fitfunc[ii][iCh]:
                        fitfunc[ii][iCh].Write()
                    if tSpectrum:
                        tSpectrum.Write()
             
    fROOTtempStorage.Close()
    
    gStyle.SetOptStat(111110)
    hist_chi2perdof = ROOT.TH1F("hist_chi2perdof", "#chi^{2}/NDF of Aging Fits; #chi^{2}/NDF; counts", 50, 0., 10.)
    
    fC1 = ROOT.TCanvas("fC1%i"%moduleNum, "fC1%i"%moduleNum, 2*1600, 2*900)
    fC1.Divide(2, 2, 0.001, 0.001)
    for ii in range(len(FEBList)):
        iFEB = FEBList[ii]
        for iCh in range(geometry_constants.nChannelPerFEB):
            fC1.Clear("D")
            
            fC1.cd(1)
            gStyle.SetOptStat(0)
            gStyle.SetOptFit(111)
            fGraphAging[ii][iCh].DrawClone("AP")
            funcAging = ROOT.TF1("funcAging_%i_%i_%i"%(moduleNum,iFEB,iCh),"[0]*TMath::Power(1.-[1]/100., x)", -0.5, 5)
            funcAging.SetParameter(0, fGraphAging[ii][iCh].GetPointY(0))
            funcAging.SetParameter(1, 5)
            funcAging.SetParName(0, "PE_0")
            funcAging.SetParName(1, "Aging Rate")
            frp = fGraphAging[ii][iCh].Fit(funcAging, "QRS")
            funcAging.SetLineColor(2) # kRed
            funcAging.DrawClone("SAME")
            if funcAging.GetNDF()>0 and funcAging.GetParameter(1)>0.:
                fHistAging[ii].Fill(funcAging.GetParameter(1))
            tPT = ROOT.TPaveText(.7, .6, .95, .9, "NDC")
            tPT.SetFillColor(0)
            tPT.AddText("Fitting with A*pow(1-r/100, x)")
            tPT.AddText("#chi^{2}/NDF = %.3f/%i"%(funcAging.GetChisquare(), funcAging.GetNDF()))
            tPT.AddText("A = %.3f #pm %.3f"%(funcAging.GetParameter(0),funcAging.GetParError(0)))
            tPT.AddText("r = %.3f #pm %.3f"%(funcAging.GetParameter(1),funcAging.GetParError(1)))
            tPT.Draw("SAME")
            
            if funcAging.GetNDF()<=0:
                print(iFEB, iCh, funcAging.GetChisquare(), funcAging.GetNDF())
            else:
                hist_chi2perdof.Fill(funcAging.GetChisquare()/float(funcAging.GetNDF()))
        
            fC1.cd(2)
            gStyle.SetOptStat(0)
            gStyle.SetOptFit(0)
            fGraphCMBTemp[ii][iCh].Draw("ALP")
            
            fC1.cd(3)
            gStyle.SetOptStat(0)
            gStyle.SetOptFit(0)
            fGrpahSPECorrected[ii][iCh].Draw("ALP")
            
            fC1.Print(pdfname, "Title: FEB%i Ch%i summary"%(iFEB, iCh))
    
    gStyle.SetOptStat(111110)
    fC = ROOT.TCanvas("csummary%i"%moduleNum, "csummary%i"%moduleNum, 2*1600, 2*900)
    fC.Divide(2, 2, 0.001, 0.001)
    
    fC.cd(1)
    gStyle.SetOptStat(0)
    gStyle.SetOptFit(0)
    fStack = ROOT.THStack("stack_aging", "Module %i aging rate; aging rate [%%]; count"%(moduleNum))
    fLegend = ROOT.TLegend(0.7, 0.7, 0.95, 0.9)
    for ii in range(len(FEBList)):
        iFEB = FEBList[ii]
        fHistAging[ii].SetLineColor(constants.rootcolors[ii])
        fStack.Add(fHistAging[ii])
        fLegend.AddEntry(fHistAging[ii],"FEB %i: #mu = %.3f, #sigma = %.3f"%(iFEB, fHistAging[ii].GetMean(), fHistAging[ii].GetRMS()))
    fStack.Draw("NOSTACK")
    fLegend.Draw("SAME")
    
    fC.cd(2)
    gStyle.SetOptStat(111110)
    gStyle.SetOptFit(0)
    gROOT.ForceStyle()
    hist_chi2perdof.Draw()
    
    fC.cd(3)
    gStyle.SetOptStat(111110)
    gStyle.SetOptFit(0)
    gROOT.ForceStyle()
    for ii in range(len(FEBList)):
        for iCh in range(geometry_constants.nChannelPerFEB):
            hist_calibAdj_sigma.Fill(np.std(np.array(SPECorrected_FEBVertical[ii][iCh])))
    hist_calibAdj_sigma.Draw()
    
    fC.cd(4)
    gStyle.SetOptStat(111110)
    gStyle.SetOptFit(0)
    gROOT.ForceStyle()
    for ii in range(len(FEBList)):
        for iCh in range(geometry_constants.nChannelPerFEB):
            hist_calibAdj_sigma_mu.Fill(np.std(np.array(SPECorrected_FEBVertical[ii][iCh]))/np.mean(np.array(SPECorrected_FEBVertical[ii][iCh])))
    hist_calibAdj_sigma_mu.Draw()
    
    fC.Print(pdfname+")", "Title: Module %i aging"%moduleNum)  
    
    fFile.Close()
    return


nWorker = 2
p = mp.Pool(processes=nWorker)
for taskspec in taskList:
    p.apply_async(evaluateModule, (taskspec[0],taskspec[1],taskspec[2]))
p.close()
p.join()
'''
for ii in range(len(taskList)):
# for ii in range(1,2):
    evaluateModule(taskList[ii][0], taskList[ii][1], taskList[ii][2])
'''

Processing Run0066_000
Processing Run1059_000
Processing Run0066_001
Processing Run1080_000
Processing Run0094_000
Processing Run1081_000
Processing Run0094_001
Processing Run0094_002
Processing Run1082_000
Processing Run0094_003
Processing Run1091_000
Processing Run0094_004
Processing Run0094_005
Processing Run1146_000
Processing Run0094_006
Processing Run0094_007
Processing Run1148_000
Processing Run0094_008
Processing Run1149_000
Processing Run0094_009
Processing Run0094_010
Processing Run0094_011
Processing Run1150_000
Processing Run0105_000
Processing Run0105_001
Processing Run0105_002
Processing Run1152_000
Processing Run0105_003
Processing Run0105_004
Processing Run1154_000
Processing Run0105_005
Processing Run0105_006
Processing Run0105_007
Processing Run1159_000
Processing Run0105_008
Processing Run0105_009
Processing Run0119_000
Processing Run1160_000
Processing Run1010_000
Processing Run1161_000
Processing Run1020_000
Processing Run1021_000
Processing Run1167_000
Processing 

'\nfor ii in range(len(taskList)):\n# for ii in range(1,2):\n    evaluateModule(taskList[ii][0], taskList[ii][1], taskList[ii][2])\n'

Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
